
This assignment is to be completed in R and returned as a clean Jupyter notebook cleared of its output. This is important otherwise Turnitin will reject the submission.

The answers require text and/or code and each one is assigned an individual cell with the message "*(Write your text here)*" or "*(Write your code here)*". **You only need to write on those cells**. 


<span style='color:Blue'> Code answers  </span>: Each part of each problem requires you to write  code that returns the output prompted when running the cell.

<span style='color:Blue'> Text answers  </span>: When required by the problem, you need to write enough text interpreting the output of the code. 

There are three problems to be tackled:
1. Plotting data and basic hypothesis testing [40%]
2. Multiple regression [40%]
3. Hierarchical modelling [20%]



- The kernel for this Jupyter notebook is R, version 4.1.0, with the following packages: MuMIn, rstanarm

In [ ]:
# You can use the following libraries for your answers
library(MuMIn)
library(rstanarm)

## Problem 1: Plotting data

You are given a dataset of ten years of Bioinformatics MSc students at Imperial College London and their end-of-year marks. Various data about the students have been collected, including: age (years), height (metres), eye colour (categorical), grey-cell brain density index (a unitless index of brain structure), the year of their graduation, their enthusiasm for the Bioinformatics course (measured in 'joys'), and their final degree percentage. All ethical approval required for imaginary data have been obtained. The data is provided in the file `q1-2021.csv`.

### Part 1

Load the dataset `q1-2021.csv`.
How many students were surveyed? How many different eye-colours have been recorded?

<span style='color:Blue'> Code Answer  </span>

In [ ]:
#part_1
#importing q1-2021.csv
q1_df <- read.csv("q1-2021.csv")
#first row is empty so we will exclude it from the rest of our analysis
q1_df <- q1_df[,-1]
#eye colour is categorical so we will save it as a factor/categorical variable
q1_df$eyecolour <- as.factor(q1_df$eyecolour)

#q1 how many students were surveyed?
part1_a <- nrow(q1_df)
#q2 how many different eye colours have been recorded?
part1_b <- length(levels(q1_df$eyecolour))


<span style='color:Blue'> Text Answer  </span>

#q1 how many students were surveyed?

$200.$

#q2 how many different eye colours have been recorded?

$\text{3 different eye colours. blue, green and green}$

### Part 2

Produce a plot to show the relationship between (1) degree mark and age and (2) degree mark and year of graduation. Do not simply use the default plotting options; alter the appearance of each plot in some way from the default (e.g., add a title, change the plotting symbol or colour).

<span style='color:Blue'> Code Answer  </span>

In [ ]:
library(ggplot2)
library(scales)
#we will plot degree marks on the y axis as it is a response variable in both cases.
plot_1 <- ggplot(data=q1_df,aes(y=mark,x=age))+
  geom_point(shape=23)+
  labs(title="Imperial College MSc Bioinformatics and Theoretical \nSystems Biology",y="Degree mark (%)",x="Age (years)")+
  theme_gray()
plot_1


plot_2  <- ggplot(data=q1_df,aes(y=mark,x=cohort))+
  geom_point(shape=1)+
  labs(title="Imperial College MSc Bioinformatics and Theoretical \nSystems Biology",y="Degree mark (%)",x="Year of Graduation")+
  theme_gray()+
  scale_x_continuous(breaks=pretty_breaks())
plot_2


---

### Part 3

Standardise the grey-cell index by age, dividing the index by age. Plot the relationship between mark and this standardised index and report whether there is a statistically significant relationship between the two indices. Report what in your code gives support for your answer.

<span style='color:Blue'> Code Answer  </span>

In [ ]:

q1_df$brain_standard <- q1_df$grey.density/q1_df$age

plot_3 <- ggplot(data=q1_df,aes(x=brain_standard,y=mark))+
  geom_point(shape=1)+
  labs(title="Imperial College MSc Bioinformatics and Theoretical \nSystems Biology",x=expression(Standardised~Grey~Cell~Brain~Density~index~(year^{-1})),y="Degree Mark (%)")
plot_3
#will use a correlation test to see if degree mark is related to the standardised index of grey.density
#pearson correlation relies upon normally distributed variables.
hist(q1_df$mark) #roughly normal
hist(log(q1_df$brain_standard)) #highly skewed to the right

#because the standardised index is right skewed we will have to use a non parametric correlation test. We will apply both Spearman's rho and Kendall's tau.
cor.test(x=q1_df$brain_standard,y=q1_df$mark,method="kendall")
cor.test(x=q1_df$brain_standard,y=q1_df$mark,method="spearman")

<span style='color:Blue'> Text Answer  </span>

No significant correlation (Spearman's $\rho=-0.053,p=0.45,n=200$) and dependence (Kendall's $\tau =-0.038,p=0.42,n=200$) is found between the standardised index for grey cell density ($y^{-1}$)  and degree mark (%). Results obtained from cor.test() using the "spearman" and "kendall" methods in R v4.2.2

---

### Part 4

Plot the relationship between mark and enthusiasm for the course, and report whether there is a statistically significant relationship between the two. Make sure to add a trend-line to your figure. Report what in your code gives support for your answer.

<span style='color:Blue'> Code Answer  </span>

In [ ]:
plot_4 <- ggplot(data=q1_df,aes(x=enthusiasm,y=mark))+
  geom_point()+
  geom_smooth(method="lm",se=FALSE)+
  labs(title="Imperial College MSc Bioinformatics and Theoretical \nSystems Biology",
       x="Enthusiasm (Joys)",
       y="Degree Mark (%)")
  
plot_4
hist(q1_df$mark) #visual inspection suggestions normal distribution
hist(q1_df$enthusiasm) #not normally distributed
#enthusiasm does not not meed the conditions to apply a pearson correlation
cor.test(x=q1_df$mark,y=q1_df$mark,method="spearman")
cor.test(x=q1_df$enthusiasm,y=q1_df$mark,method="kendall")


<span style='color:Blue'> Text Answer  </span>

A significant correlation (Spearman's $\rho=1,p<0.001,n=200$) and dependence (Kendall's $\tau =0.371,p<0.001,n=200$) is found between Enthusiasm (joys) and degree mark (%). Results obtained from cor.test() using the "spearman" and "kendall" methods in R v4.2.2

---

## Problem 2: Multiple regression

You have been given a dataset on the effective reproduction number ('Rt') of SARS-CoV-2 early in the pandemic across regions of the US in June 2020. This number can go as low as 0 in regions where there is essentially no transmission, and numbers greater than 1 indicate a growing population of SARS-CoV-2. The regions (their names and locations are unimportant) have associated data: temperature (in degrees Centigrade), humidity (percentage saturation of the air with water), population density (mean number of people per square kilometer), and non-pharmaceutical intervention strength (a dimensionless, normalised index of lockdown strength and mask compliance). While these data are simulated, they are based on a real-world exercise that researchers at Imperial carried out early in the pandemic (see Smith et al. 2021 PNAS 118 (25) e2019284118).

### Part 1

Load the data into R from the file `q2-2021.csv`. Fit an additive model with Rt as the response variable and all other variables as explanatory variables. Summarise briefly what this model shows.

<span style='color:Blue'> Code Answer  </span>

In [ ]:
### Part 1

###

q2_df <- read.csv("q2-2021.csv")
#first collumn is unneccesary
q2_df <- q2_df[,-1]

additive_model <- lm(data=q2_df,formula=Rt~temp+humid+pop.dens+exposure)
summary(additive_model)


<span style='color:Blue'> Text Answer  </span>


The additive model explains significantly more variation in $R_t$ than a null model with $R_t \sim  \mu$. ($F_{4,495}=312.3,\, p<0.001,\, R^2=0.7162,\, R^2_{adj}=0.7139$). All parameters significantly predicted $R_{t}$.


| Parameter          | Estimate  | Std.error | t value   | Pr(>\|t\|) |
|--------------------|-----------|-----------|-----------|------------|
| Intercept          | 3.22E+00  | 5.41E-01  | 5.95E+00  | 4.97E-09   |
| Temperature        | -9.49E-02 | 2.69E-02  | -3.53E+00 | 4.53E-04   |
| Humidity           | 1.16E-02  | 9.19E-04  | 1.26E+01  | < 2e-16    |
| Population density | 8.82E-06  | 1.24E-06  | 7.09E+00  | 4.66E-12   |
| NPI-strength       | -8.44E-01 | 2.68E-02  | -3.15E+01 | < 2e-16    |

---

### Part 2

There are good reasons from theory to suppose that the impact of population density and non-pharmaceutical interventions are non-linear. Apply a log10 transformation to the population density data and a logistic transformation (function definition given below so that it can be used analogously to the `log10` function in R) to your variables and use ANOVA to test for an improvement in model fit after these transformations. Note that, because there will be no difference in the number of parameters in your model, you will not be able to generate a probability (p-value) for this test. Thus you will need to interpret the F-statistic itself and use your statistical judgement. Briefly provide your reasoning in your answer.

`logistic <- function(x) 1/(1+exp(-x))`

<span style='color:Blue'> Code Answer  </span>

In [ ]:
#Part 2
logistic <- function(x) 1/(1+exp(-x))
q2_df$log_Pop_density <- log10(q2_df$pop.dens)
q2_df$logit_npi <- logistic(q2_df$exposure)
transf_additive_model <- lm(data=q2_df,formula=Rt~temp+humid+log_Pop_density+logit_npi)
summary(transf_additive_model)
#line below will not generate an F statistic
anova(transf_additive_model,additive_model)

<span style='color:Blue'> Text Answer  </span>

(Write your text here)
Because the purely additive model and the transformed additive model have the same number of parameters ($5$). An F test to see if the transformed additive model explains more variation than the purely additive model cannot be performed.

$$
F=\frac{(SSM_{a}-SSM_{n}\times (n-k_{a}-1))}{SSE \times k_{a}-k_{n}}
$$

Because $k_{a}=k_{b}=5.$ $k_{a}-k_{n}=0$ and no F statistic can be calculated. Instead we will calculate the $F$ statistic of the transformed model against the null model $R_{t} \sim \mu$

$$F_{4,495}=407.6,\, p<0.001, \, R^{2}=0.7652$$

We reccomend accepting the transformed model as the ratio of variation explained by the model $SSM$ to $SSE$ is $1.3\times$ greater in the transformed model. $\frac{407.6}{312.3}=1.31$. In more intuitive terms, a greater amount of variation is explained by the model while estimating the same amount of parameters.

---


### Part 3

Identify the most important drivers of transmission (Rt) in this dataset and quantify their relative importances.

<span style='color:Blue'> Code Answer  </span>

In [ ]:
#part 3
#standardising explanatory variables
q2_df_scaled <- scale(q2_df[,-5]) #NB not standardising the response variable Rt.
q2_df_scaled <- as.data.frame(q2_df_scaled) #scale produces a matrix, changing to data frame
q2_df_scaled$Rt <- q2_df$Rt
#building transformed_additive_model
transf_additive_model_scaled <- lm(data=q2_df_scaled,formula=Rt~temp+humid+log_Pop_density+logit_npi)
#Reference: Groempig et al 2007 DOI:https://www.jstatsoft.org/article/view/v017i01
#Within this paper reference is made to two ways of quantifying importance of a variable in a linear model. The dispersion importance relating to the explained sum of squares/ R^2 as can be done in the library relaimpo and the level importance relating to the effect size of a variable on the mean of the response variable.
# we will be calculating relative importance using standardised variables and the standard effect sizes as the question asks for the important *drivers* of Rt as opposed to the *most explanatory* variables.
summary(transf_additive_model_scaled)


<span style='color:Blue'> Text Answer  </span>


The drivers of $R_{t}$ in descending order of importance is the $\text{logit}(\text{NPI})$  which has a  $0
.85/0.33=2.93 \times$ greater effect than $\text{humidity}$ followed by $\log_{10}(\text{Population Density})$ which is $0.29/0.33=0.879 \times$ weaker than $\text{humidity}$ followed by $\text{temperature}$ which is $0.09/0.29=0.310 \times $ weaker than $\log_{10}(\text{Population Density})$. 

|          Parameter         | Effect Size |
|:--------------------------:|:-----------:|
|          Intercept         |     2.01    |
|         Temperature        |    -0.09    |
|          Humidity          |     0.33    |
|  log10(Population density) |     0.29    |
|     logit(NPI-strength)    |    -0.85    |

---

### Part 4

There are strong theoretical grounds to suppose that the impact of non-pharameutical interventions mediate the impact of all other variables in this analysis. Statistically test for evidence that non-pharmaceutical interventions interact with all other variables in this analysis.

<span style='color:Blue'> Code Answer  </span>

In [1]:
library(MuMIn)
# Load package (named for Moomin cartoons)

#now building top model using transformed variables
top_standard_model <- lm(data=q2_df_scaled,formula=Rt~(temp+humid+log_Pop_density+logit_npi*temp+logit_npi*humid+logit_npi*log_Pop_density),na.action = na.pass)
# 'dredging' through all the subsets
models <- dredge(top_standard_model)
# Subset our models to contain only those within 4 AIC units of the best model
# - this is the standard number, but there's no reason to use one over another really
# - then summarize across the average of these models
summary(model.avg(models, subset=delta<4, fit=TRUE))
# Get the AIC importance values (the 'summed weights')
sw(model.avg(models, subset=delta<4, fit=TRUE))


UndefVarError: UndefVarError: library not defined

<span style='color:Blue'> Text Answer  </span>

Out of all subsets of interactions between $\text{logit}(\text{NPI})$ and the other variables within $4 \text{AIC}$ units of the best model, 2 models were obtained. The best performing model includes all variables and interactions and the second model differs in that it excludes an interaction between $\text{logit}(\text{NPI})$ and $\text{temperature}$. The models perform highly similarly with very similar AICs.

|  Component model  | log likelihood |  AICc  | delta AIC | weight |
|:-----------------:|:--------------:|:------:|:---------:|:------:|
|     All terms     |     -356.92    | 732.22 |     0     |  0.55  |
|  -logit(NPI):temp |     -358.18    | 732.65 |    0.44   |  0.45  |

The effect size of the interaction terms between $\text{logit}(\text{NPI})$ are of comparable effect size to $\text{temperature}$ except for the interaction between $\text{temperature}$ and $\text{logit}(\text{NPI})$ and $\text{temperature}$ which is much smaller due to the unconditional averaging. Even in the model where it is present the effect size is notably small $\text{Estimate}= 0.036$.

|     Model   Averaged Coefficients    | Estimate | z value | Pr(>\|z\|) |
|:------------------------------------:|:--------:|:-------:|:----------:|
|               Intercept              |  2.0125  |  90.051 |   <2E-16   |
|               humidity               |  0.32227 |  14.305 |   <2E-16   |
|        log(Population density)       |  0.29171 |  12.985 |   <2E-16   |
|              logit(NPI)              | -0.85415 |  38.148 |   <2E-16   |
|              temperature             | -0.10147 |  4.524  |  6.10E-06  |
|          humidity:logit(NPI)         | -0.10836 |  4.952  |  7.00E-07  |
| log(Population   density):logit(NPI) |  -0.1459 |  6.579  |   <2E-16   |
|        logit(NPI):temperature        |  0.01996 |  0.807  |    0.419   |

Finally looking at the summed weights it seems clear that $\text{logit}(\text{NPI})$:$\text{temperature}$ does not have as much support for its inclusion in our final model.
This is not to claim that the interaction is not important or informative as it does lower the AIC of the model containing it. However, given its low effect size and little loss in AIC we reccomend picking the simpler model.

|                                      | Sum of weights | # of models containing parameter |
|:------------------------------------:|:--------------:|:--------------------------------:|
|               humidity               |      1.00      |                 2                |
|        log(Population density)       |      1.00      |                 2                |
|              logit(NPI)              |      1.00      |                 2                |
|              temperature             |      1.00      |                 2                |
|          humidity:logit(NPI)         |      1.00      |                 2                |
| log(Population   density):logit(NPI) |      1.00      |                 2                |
|        logit(NPI):temperature        |      0.55      |                 2                |
---

## Problem 3: Hierarchical modelling

You are given a dataset (`q3-2021.csv`) on scream intensity (decibels) and statistics exam performance (%), generated by a well-known statistical consultancy company (https://xkcd.com/2533/). Fit an appropriate model to these data to test whether scream intensity is associated with exam performance. Make sure to clearly explain why you chose your particular statistical formulation, and to reference whether you find, or do not find, support for the hypothesis with reference to your data.

<span style='color:Blue'> Code Answer  </span>

In [ ]:
q3_df <- read.csv("q3-2021.csv")
#first collumn unneccesary
q3_df <- q3_df[,-1]


library(ggplot2)
ggplot(data=q3_df,aes(x=scream,y=grade,col=student))+
  geom_point()+geom_smooth(method="lm")+
  labs(title="STATS EXAM GRADE",x="SCREAM LOUDNESS (DECIBELS)",y="GRADE (%)")

library(rstanarm)
model <- stan_glmer(grade ~ scream + (1|student),data=q3_df)


# More comprehensive and useful coefficients
summary(model, probs=c(.025, .5, .975))
# "Caterpillar" coefficient plots
plot(model)
# MCMC diagnostics (see note in text)
plot(model, "trace")

# area plots
# all of the parameters
# scale of parameters is so varied
# that it's hard to see the details
library(bayesplot)
sims <- as.matrix(model)
plot_title <- ggtitle("Posterior distributions",
                      "with medians and 95% intervals")
mcmc_areas(sims, prob = 0.9) + plot_title

# area plot for scream parameter
mcmc_areas(sims,
           pars = c("scream"),
           prob = 0.9) + plot_title

posterior_interval(model,pars="scream",prob=0.53) #23.5 % of distribution below 0

<span style='color:Blue'> Text Answer  </span>

It is clear from the plots of Grade against Scream Loudness that the student screaming highly influences both the scream and the grade achieved. A linear model would be "swamped" by the explanatory power of the student and would not reliably test the relationship between scream and grade. We suggest a heirarchical model where the student varies the intercept of the grade parameter. We chose to implement this heirarchical model in a Bayesian framework as this allows us to visualise a posterior distribution for the scream parameter. Addtionally, mixed effects models do not allow for testing the significance of any fixed effect or calculating the $r^2$.

We ran a MCMC Bayesian heirarchical model using rstanarm running a total of 4 chains of 2000 iterations sampling every $200$ iterations. The models were checked graphically for convergence and $\hat r$ were all equal to 1. We applied normally distributed priors with a $\mu$ of $0$ so as to be uninformative.

Parameter Estimates and their posterior distributions are summarised in the penultimate plot. The posterior distribution for the scream parameter is found in the final plot and is found to lie with 95% probability within the left skewed interval $[-0.5,1.4]$ with a mean of $0.3$. We conclude that there is insufficient evidence for screams to be associated with statistics grades even when controlling for variation between students because the 95% posterior interval contains 0 and indeed  $\sim25\%$ of the distribution (last line) is $<0$.